In [52]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adam

In [53]:
# Read data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [54]:
# Remove un-use columns

def drop_columns(df, list_columns=[]):
  return df.drop(list_columns, axis =1 )

list_columns =  ["PassengerId", "Name", "Ticket", "Fare", "Cabin", "Embarked"]
train_data = drop_columns(train_data, list_columns)
list_columns_test =  ["PassengerId", "Name", "Ticket", "Fare", "Cabin", "Embarked"]
test_data = drop_columns(test_data, list_columns_test)


In [55]:
# Remove nan in exist columns

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [56]:
# Split purpose

train_data_y = train_data.Survived
list_columns_survived =  ["Survived"]
train_data = drop_columns(train_data, list_columns_survived)

In [57]:
# one-hot vector

def one_hot_vector(df, list_OHV_columns):
  for column_name in list_OHV_columns:
    df = pd.get_dummies(df, columns=[column_name], prefix="oh"+column_name, sparse=True)
  return df

one_hot_columns = ["Pclass"]
train_data = one_hot_vector(train_data, one_hot_columns)
test_data = one_hot_vector(test_data, one_hot_columns)


In [58]:
# label encoding
sex_le = LabelEncoder()
train_data.Sex = sex_le.fit_transform(train_data.Sex)
test_data.Sex = sex_le.fit_transform(test_data.Sex)

In [59]:
# Standardization

sc = StandardScaler()
train_data.Age = sc.fit_transform(train_data.Age.values.reshape(-1, 1))
test_data.Age = sc.transform(test_data.Age.values.reshape(-1,1))

In [60]:
# to numpy
X = train_data.values
y = train_data_y.values

In [61]:
X

array([[ 1.        , -0.53037664,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        ,  0.57183099,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        , -0.25482473,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.        , -0.73704057,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        , -0.25482473,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.15850313,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [62]:
# Split data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [63]:
# create model

model = Sequential()
model.add(Dense(20, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, input_dim=20))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.0001)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

c:\Users\0720k\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [64]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [68]:
model.fit(X_train, y_train, epochs=1000, batch_size=32)

Epoch 1/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7566 - accuracy: 0.4869
Epoch 2/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7495 - accuracy: 0.4676
Epoch 3/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7264 - accuracy: 0.4956
Epoch 4/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7212 - accuracy: 0.5289
Epoch 5/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7357 - accuracy: 0.5149
Epoch 6/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7302 - accuracy: 0.5044
Epoch 7/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7229 - accuracy: 0.5131
Epoch 8/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7260 - accuracy: 0.5026
Epoch 9/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7010 - accuracy: 0.5394
Epoch 10/1000
18/18 [==============================] - 0s 2ms/step - loss: 0.7454 - accuracy: 0.4886

In [69]:
score = model.evaluate(X_train, y_train)
print("")
print("Test loss:{0}".format(score[0]))
print("Test accuracy:{0}".format(score[1]))

18/18 [==============================] - 0s 2ms/step - loss: 0.6124 - accuracy: 0.7075

Test loss:0.6124234795570374
Test accuracy:0.707530677318573
